# <div style="padding:2rem;font-size:100%;text-align:left;display:fill;border-radius:0.25rem;overflow:hidden;background-image: url(https://images.pexels.com/photos/2860804/pexels-photo-2860804.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1)"><b><span style='color:grey'> PARKING ANALYSIS PREDICTOR</span></b> </div>

This is a collaborative group project done at the end of Phase 4 of Moringa School's Data Science program. The team members of this group include:
1. [Mwiti Mwongo](https://github.com/M13Mwongo)
2. [Grace Mutuku](https://github.com/GraceKoki)
3. [Joy Ogutu](https://github.com/Ogutu01)
4. [Ezra Kipchirchir](https://github.com/dev-ezzy)
5. [Mary Gaceri](https://github.com/MaryGaceri)

## <div style="padding:2rem;font-size:80%;text-align:left;display:fill;border-radius:0.25rem;overflow:hidden;background-image: url(https://images.pexels.com/photos/2860804/pexels-photo-2860804.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1)"><b><span style='color:white'> Problem statement</span></b> </div>

Traffic is a nightmare, am I right? You can’t drive anywhere without being stuck in traffic for a while, especially in Nairobi. What makes it worse is that a lot of times during high-traffic periods, such as the mornings and evenings, there is a high likelihood of missing out on your desired parking spot that is near your office, especially when looking at county-run parking.
This application hopes to predict the parking patterns and likelihood of having available parking spots in certain areas at a given time of the day.


## <div style="padding:2rem;font-size:80%;text-align:left;display:fill;border-radius:0.25rem;overflow:hidden;background-image: url(https://images.pexels.com/photos/2860804/pexels-photo-2860804.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1)"><b><span style='color:white'> Objectives</span></b> </div>

### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#d4ff00;height:35px;width:4px;margin:0 1rem 0 0;border-radius:2rem'/><span>Main Objective:</span></div>

- Predict the availability of parking in a given area in Nairobi

## <div style="padding:2rem;font-size:80%;text-align:left;display:fill;border-radius:0.25rem;overflow:hidden;background-image: url(https://images.pexels.com/photos/2860804/pexels-photo-2860804.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1)"><b><span style='color:white'>Proposed Solution</span></b> </div>

Here we created a model that would predict the availability of parking spots across Nairobi. Our procedure for doing this is as follows, starting with the data sourcing and culminating in the model creation and deployment.

### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#d4ff00;height:35px;width:4px;margin:0 1rem 0 0;border-radius:2rem'/><span>**0. Preliminaries**</span></div>

#### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#9DF7E5;height:35px;width:3px;margin:0 1rem 0 0;border-radius:2rem'/><span>**a) Imports & OOP**</span></div>

The necessary libraries were first imported.

In [ ]:
# Importing necessary libraries
# Basics
import pandas as pd
import numpy as np
import itertools
from io import StringIO
from datetime import datetime, timedelta
from requests import api

# Visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline 
import plotly.express as px
import seaborn as sns
import matplotlib.patches as mpatches
from matplotlib.pylab import rcParams
import time

# Modeling libraries
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA        
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf, adfuller
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import TimeSeriesSplit 
from pmdarima import auto_arima      

from prophet import Prophet 

#Model deployment libraries
import joblib    


# Warnings
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.filterwarnings('ignore')

# Custom Options for displaying rows.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',100)

In object-oriented programming (OOP), classes serve as blueprints, dividing code into modular components that contain data and actions. They encourage encapsulation, abstraction, and inheritance in code, making it more modular, readable, and manageable. Classes offer an organized way to developing and implementing code, promoting clarity and efficiency in software development.

Consequently, the following classes were implemented and defined below:
1. Data Sourcing
2. Data Preprocessing
3. Data Analysis
4. Modelling

In [ ]:
class DataSourcing:
  def __init__(self,carparks_file_path,carpark_details_file_path,carpark_history_file_path):
    
    df_carparks = pd.read_json(carparks_file_path)
    df_carpark_details = pd.read_json(carpark_details_file_path)
    df_carpark_history = pd.read_json(carpark_history_file_path)
    
    self.carparks_all = df_carparks
    self.carpark_details = df_carpark_details
    self.carpark_history = df_carpark_history
  
  def get_carparks(self):
    """
    Get the carparks from the instance variable `carparks_all`.
    
    Returns:
        list: List of carparks.
    """
    return self.carparks_all
  def get_carpark_details(self):
    """
    Get the carpark details.

    Returns:
        dict: The carpark details.
    """
    return self.carpark_details
  def get_carpark_history(self):
    """
    Get the car park history.

    Returns:
        carpark_history: The history of the car park.
    """
    return self.carpark_history
  
class DataPreprocessing:
  pass

class DataAnalysis:
  pass

class Modelling:
  pass

#### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#9DF7E5;height:35px;width:3px;margin:0 1rem 0 0;border-radius:2rem'/><span>**b) The Process of Fetching Data**</span></div>

Our data was sourced from the Transport for New South Wales(TfNSW) website, more speficially, from their [Car Park API](https://opendata.transport.nsw.gov.au/dataset/car-park-api).

The API - whose base URL was `https://api.transport.nsw.gov.au/v1/carpark` - had two endpoints:
1. `{baseURL}?facility={facility_id}` - Containts one optional variable ***facility_id***. Returns occupancy details of a car park based on a facility ID. If the facility ID specified, a list of facility names with their ID will be returned.
2. `{baseURL}}/history?facility={facility_id}&eventdate={date_in_question}` - Contains two mandatory variables, ***facility_id*** and ***date_in_question*** formatted as *YYYY-MM-DD*. Returns historical occupancy details of a car park based on a facility ID
and event date. 

Our intention was to use this API to fetch six months' worth of historical parking data. An extensive time period would lead to a proper understanding of parking habits across a wide array of conditions while factoring in social events, public holidays, school holidays and even leave days of employees.

The team came up with code to automatically make requests to the API, and save this information in a dataframe. However, after further study of the API's structure and the data being received, the team saw it best to have these requests made once and the resulting data stored in json files, which can be read by pandas.

The function below was used to retrieve car park data from the TfNSW API and saves it to a JSON file.

```python
def get_carparks_list():
  import dotenv
  import os
  
  # Load secret file with api key
  dotenv.load_dotenv('.env')
  
  headers = {
    "Authorization":f"apikey {os.environ.get('apikey')}"
  }
  url_carparks = 'https://api.transport.nsw.gov.au/v1/carpark'
  
  list_of_carparks = requests.get(url_carparks, headers=headers).json()
  
  # Create a json file and write the data if it doesn't exist
  if not os.path.exists('./data/carparks.json'):
    with open('./data/carparks.json', 'w') as f:
      json.dumps(list_of_carparks, f)
  
  # The format of the file itself is a dictionary, thus requiring some creativity to read
  with open('./data/carparks.json') as f:
    data = json.load(f)
  
  # Convert the dictionary to a dataframe
  df_carparks = pd.DataFrame.from_dict(data, orient='index')
  # Resetting the index to label the columns afterwards
  df_carparks = df_carparks.reset_index()
  df_carparks.columns = ['facility_id', 'CarParkName']
  
  df_carparks
```

### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#d4ff00;height:35px;width:4px;margin:0 1rem 0 0;border-radius:2rem'/><span>**1. Data Sourcing:**</span></div>

### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#d4ff00;height:35px;width:4px;margin:0 1rem 0 0;border-radius:2rem'/><span>**2. Data Understanding:**</span></div>

### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#d4ff00;height:35px;width:4px;margin:0 1rem 0 0;border-radius:2rem'/><span>**3. Data Preprocessing:**</span></div>

### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#d4ff00;height:35px;width:4px;margin:0 1rem 0 0;border-radius:2rem'/><span>**4. Data Handling**</span></div>

### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#d4ff00;height:35px;width:4px;margin:0 1rem 0 0;border-radius:2rem'/><span>**5. Explorative Data Analysis & Visualisation**</span></div>

### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#d4ff00;height:35px;width:4px;margin:0 1rem 0 0;border-radius:2rem'/><span>**6. Modelling**</span></div>

### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#d4ff00;height:35px;width:4px;margin:0 1rem 0 0;border-radius:2rem'/><span>**7. Deployment**</span></div>

### <div style='display:flex;align-items:center;flex-direction:row'><hr style='background-color:#d4ff00;height:35px;width:4px;margin:0 1rem 0 0;border-radius:2rem'/><span>**8. Conclusion**</span></div>

## <div style="padding:2rem;font-size:80%;text-align:left;display:fill;border-radius:0.25rem;overflow:hidden;background-image: url(https://images.pexels.com/photos/2860804/pexels-photo-2860804.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1)"><b><span style='color:white'>Conclusion</span></b> </div>